In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import networkx as nx
import plotly.graph_objects as go
import numpy as np

In [ ]:
# def read_graph_from_txt(file_path):
#     nodes = set()
#     edges = []
    
#     with open(file_path, 'r') as file:
#         lines = file.readlines()
        
#         # Skip the header lines starting with '#'
#         for line in lines:
#             if not line.startswith('#'):
#                 break
        
#         # Read the edges
#         for line in lines:
#             if not line.startswith('#'):
#                 from_node, to_node = map(int, line.strip().split())
#                 nodes.add(from_node)
#                 nodes.add(to_node)
#                 edges.append((from_node, to_node))
    
#     return nodes, edges

In [ ]:
# nodes, edges = read_graph_from_txt('aa.txt')

In [ ]:
# print("Number of Nodes:", len(nodes))
# print("Number of Edges:", len(edges))

In [ ]:
import networkx as nx
from node2vec import Node2Vec
import numpy as np
from sklearn.metrics import roc_auc_score

# Read the graph from a .txt file
def read_graph_from_txt(file_path):
    nodes = set()
    edges = []

    with open(file_path, 'r') as file:
        lines = file.readlines()

        # Read the edges
        for line in lines:
            if not line.startswith('#'):
                from_node, to_node = map(int, line.strip().split())
                nodes.add(from_node)
                nodes.add(to_node)
                edges.append((from_node, to_node))

    return nodes, edges

In [ ]:
# import networkx as nx
# import matplotlib.pyplot as plt
# import random
# import numpy as np
# from sklearn.decomposition import PCA

In [ ]:
# def load_sample_graph():
#     num_nodes = 100
#     avg_degree = 10
#     rewiring_prob = 0.1
#     G = nx.watts_strogatz_graph(num_nodes, avg_degree, rewiring_prob)
    
#     return G

In [ ]:
# # Load the graph and plot it
# graph = load_sample_graph()

In [ ]:
# Perform link prediction using Node2Vec embeddings
def link_prediction(graph, embedding_dimension=64, walk_length=30, num_walks=200, window_size=10):
    node2vec = Node2Vec(graph, dimensions=embedding_dimension, walk_length=walk_length, num_walks=num_walks, workers=4)
    model = node2vec.fit(window=window_size, min_count=1, batch_words=4)

    # Generate embeddings for all nodes
    node_embeddings = {node: model.wv[node] for node in graph.nodes()}

    # Perform link prediction
    link_predictions = []
    for edge in graph.edges():
        source, target = edge
        if source not in node_embeddings or target not in node_embeddings:
            continue
        source_embedding = node_embeddings[source]
        target_embedding = node_embeddings[target]
        similarity = np.dot(source_embedding, target_embedding) / (np.linalg.norm(source_embedding) * np.linalg.norm(target_embedding))
        link_predictions.append((source, target, similarity))

    return link_predictions

In [ ]:
graph_file_path = "aa.txt"  # Replace with the path to your .txt file
nodes, edges = read_graph_from_txt(graph_file_path)

In [ ]:
# Create a graph
graph = nx.Graph()
graph.add_nodes_from(nodes)
graph.add_edges_from(edges)

In [ ]:
link_predictions = link_prediction(graph)


In [ ]:
# Extract true labels and predicted scores for both positive and negative examples
true_labels = []
predicted_scores = []
for edge in graph.edges():
    source, target = edge
    true_labels.append(1)
    predicted_scores.append(next((similarity for src, tgt, similarity in link_predictions if (src == source and tgt == target) or (src == target and tgt == source)), 0.0))

# Generate negative examples
num_negative_examples = len(true_labels)
while len(true_labels) < 2 * num_negative_examples:
    non_edge = np.random.choice(list(graph.nodes()), size=2, replace=False)
    if not graph.has_edge(*non_edge):
        true_labels.append(0)
        predicted_scores.append(next((similarity for src, tgt, similarity in link_predictions if (src == non_edge[0] and tgt == non_edge[1]) or (src == non_edge[1] and tgt == non_edge[0])), 0.0))

true_labels = np.array(true_labels)
predicted_scores = np.array(predicted_scores)

In [ ]:
# Calculate AUC score
auc_score = roc_auc_score(true_labels, predicted_scores)
print(f"AUC Score: {auc_score:.4f}")

In [ ]:
import networkx as nx
import random
from gensim.models import Word2Vec
import networkx as nx
import random
from gensim.models import Word2Vec
import matplotlib.pyplot as plt

class Node2Vec:
    def __init__(self, graph, p, q, num_walks, walk_length, dimensions):
        self.graph = graph
        self.p = p  # Return parameter
        self.q = q  # In-out parameter
        self.num_walks = num_walks
        self.walk_length = walk_length
        self.dimensions = dimensions
        
    def simulate_walks(self, node):
        walks = []
        for _ in range(self.num_walks):
            walk = [node]
            while len(walk) < self.walk_length:
                current_node = walk[-1]
                neighbors = list(self.graph.neighbors(current_node))
                if len(neighbors) > 0:
                    if len(walk) == 1:
                        walk.append(random.choice(neighbors))
                    else:
                        prev_node = walk[-2]
                        next_node = self.sample_next_node(current_node, prev_node)
                        walk.append(next_node)
                else:
                    break
            walks.append(walk)
        return walks
    
    def sample_next_node(self, current_node, prev_node):
        neighbors = list(self.graph.neighbors(current_node))
        weights = [1.0 if n == prev_node else 1.0/self.q if n in self.graph.neighbors(prev_node) else 1.0/self.p for n in neighbors]
        return random.choices(neighbors, weights=weights)[0]
    
    # def learn_embeddings(self, walks):
    #     model = Word2Vec(walks, size=self.dimensions, window=5, min_count=0, sg=1, workers=4, iter=1)
    #     return model
    def learn_embeddings(self, walks):
        model = Word2Vec(walks, vector_size=self.dimensions, window=5, min_count=0, sg=1, workers=4, epochs=1)
        return model
    
    def run(self):
        walks = []
        nodes = list(self.graph.nodes())
        for node in nodes:
            walks.extend(self.simulate_walks(node))
        
        model = self.learn_embeddings(walks)
        return model




In [ ]:
# Create a graph
G = nx.karate_club_graph()


In [ ]:
# Find the number of nodes and edges
num_nodes = G.number_of_nodes()
num_edges = G.number_of_edges()

In [ ]:
# Find the number of nodes and edges
num_nodes = G.number_of_nodes()
num_edges = G.number_of_edges()
print("Number of nodes:", num_nodes)
print("Number of edges:", num_edges)

In [ ]:
# Create a new figure and axis
fig, ax = plt.subplots(figsize=(8, 6))

# Visualize the original graph
pos = nx.spring_layout(G)
nx.draw(G, pos, with_labels=True, node_color='skyblue', node_size=500, ax=ax)
ax.set_title("Original Graph")
plt.show()
plt.close()

In [ ]:
# Initialize node2vec with parameters
p = 1.0
q = 1.0
num_walks = 10
walk_length = 4
dimensions = 128

In [ ]:
node2vec = Node2Vec(G, p, q, num_walks, walk_length, dimensions)
model = node2vec.run()

In [ ]:
# Get node embeddings
node_embeddings = {str(node): model.wv[str(node)] for node in G.nodes()}

In [ ]:
embedding.shape

In [ ]:
print("Embedding for node", 2, ":", embedding[2])

In [ ]:
print("Embedding for node", 7, ":", embedding)

In [ ]:
import networkx as nx
from node2vec import Node2Vec
import matplotlib.pyplot as plt

In [ ]:
# Create a sample graph (you can replace this with your own graph)
G = nx.karate_club_graph()



In [ ]:
# Create a new figure and axis
fig, ax = plt.subplots(figsize=(8, 6))

# Visualize the original graph
pos = nx.spring_layout(G)
nx.draw(G, pos, with_labels=True, node_color='skyblue', node_size=500, ax=ax)
ax.set_title("Original Graph")
plt.show()
plt.close()

In [ ]:
# Precompute probabilities and generate walks
node2vec = Node2Vec(G, dimensions=64, walk_length=30, num_walks=200, p=0.5, q=2, workers=4)

# Embed nodes
model = node2vec.fit(window=10, min_count=1)

# Get node embeddings
node_embeddings = {str(node): model.wv[str(node)] for node in G.nodes()}


In [ ]:
# Visualize the embeddings using scatter plot
plt.figure(figsize=(20, 15))
for node, embedding in node_embeddings.items():
    plt.scatter(embedding[0], embedding[1], color='blue')
    plt.annotate(node, (embedding[0], embedding[1]), alpha=0.5)
plt.title("Node2Vec Embeddings")
plt.xlabel("Dimension 1")
plt.ylabel("Dimension 2")
plt.show()

In [ ]:
# import numpy as np
# import networkx as nx
# from gensim.models import Word2Vec
# from typing import List, Dict
# from sklearn.cluster import KMeans
# from sklearn.manifold import TSNE
# import matplotlib.pyplot as plt

# class AdvancedNode2Vec:
#     def __init__(
#         self,
#         graph: nx.Graph,
#         p: float,
#         q: float,
#         dimensions: int,
#         num_walks: int,
#         walk_length: int,
#         window_size: int,
#         num_epochs: int,
#         workers: int
#     ) -> None:
#         self.graph = graph
#         self.p = p
#         self.q = q
#         self.dimensions = dimensions
#         self.num_walks = num_walks
#         self.walk_length = walk_length
#         self.window_size = window_size
#         self.num_epochs = num_epochs
#         self.workers = workers
        
#         self.walks = self.generate_walks()
#         self.model = self.train_embedding()

#     def generate_walks(self) -> List[List[str]]:
#         walks = []
#         for _ in range(self.num_walks):
#             for node in self.graph.nodes():
#                 walk = self.node2vec_walk(node)
#                 walks.append(walk)
#         return walks

#     def node2vec_walk(self, start_node: str) -> List[str]:
#         walk = [start_node]
#         while len(walk) < self.walk_length:
#             curr_node = walk[-1]
#             neighbors = list(self.graph.neighbors(curr_node))
#             if len(neighbors) > 0:
#                 if len(walk) == 1:
#                     walk.append(np.random.choice(neighbors))
#                 else:
#                     prev_node = walk[-2]
#                     next_node = self.weighted_choice(neighbors, prev_node, curr_node)
#                     walk.append(next_node)
#             else:
#                 break
#         return walk

#     def weighted_choice(self, neighbors: List[str], prev_node: str, curr_node: str) -> str:
#         weights = []
#         for neighbor in neighbors:
#             if neighbor == prev_node:
#                 weights.append(1 / self.p)
#             elif self.graph.has_edge(curr_node, neighbor):
#                 weights.append(1)
#             else:
#                 weights.append(1 / self.q)
#         weights = np.array(weights)
#         weights /= weights.sum()
#         return np.random.choice(neighbors, p=weights)

#     def embed_all_nodes(self) -> Dict[str, np.ndarray]:
#         embeddings = {}
#         for node in self.graph.nodes():
#             embeddings[node] = self.model.wv[node]
#         return embeddings

#     def train_embedding(self) -> Word2Vec:
#         model = Word2Vec(
#             sentences=self.walks,
#             vector_size=self.dimensions,
#             window=self.window_size,
#             sg=1,  # Skip-gram
#             epochs=self.num_epochs,  
#             workers=self.workers
#         )
#         return model
    
#     def get_node_similarity(self, node1: str, node2: str) -> float:
#         return self.model.wv.similarity(node1, node2)
    
#     def get_node_clusters(self, num_clusters: int) -> Dict[int, List[str]]:
#         node_embeddings = [self.model.wv[node] for node in self.graph.nodes()]
#         kmeans = KMeans(n_clusters=num_clusters)
#         cluster_assignments = kmeans.fit_predict(node_embeddings)

#         clusters = {}
#         for node, cluster_id in zip(self.graph.nodes(), cluster_assignments):
#             if cluster_id not in clusters:
#                 clusters[cluster_id] = []
#             clusters[cluster_id].append(node)

#         return clusters
    
#     def visualize_embeddings(self, nodes_to_visualize: List[str]) -> None:
#         node_embeddings = [self.model.wv[node] for node in nodes_to_visualize]
#         tsne = TSNE(n_components=2)
#         embeddings_2d = tsne.fit_transform(node_embeddings)

#         plt.figure(figsize=(10, 8))
#         plt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1])
#         for i, label in enumerate(nodes_to_visualize):
#             plt.annotate(label, (embeddings_2d[i, 0], embeddings_2d[i, 1]))
#         plt.title("Node Embeddings Visualization")
#         plt.show()


In [ ]:
class Node2Vec:
    def __init__(self, graph, p, q, dimensions, num_walks, walk_length, window_size, num_epochs, workers):
        self.graph = graph
        self.p = p
        self.q = q
        self.dimensions = dimensions
        self.num_walks = num_walks
        self.walk_length = walk_length
        self.window_size = window_size
        self.num_epochs = num_epochs
        self.workers = workers
        
        self.walks = self.generate_walks()
        self.model = self.train_embedding()

    def generate_walks(self):
        walks = []
        for _ in range(self.num_walks):
            for node in self.graph.nodes():
                walk = self.node2vec_walk(node)
                walks.append(walk)
        return walks

    def node2vec_walk(self, start_node):
        walk = [start_node]
        while len(walk) < self.walk_length:
            curr_node = walk[-1]
            neighbors = list(self.graph.neighbors(curr_node))
            if len(neighbors) > 0:
                if len(walk) == 1:
                    walk.append(np.random.choice(neighbors))
                else:
                    prev_node = walk[-2]
                    next_node = self.weighted_choice(neighbors, prev_node, curr_node)
                    walk.append(next_node)
            else:
                break
        return walk

    def weighted_choice(self, neighbors, prev_node, curr_node):
        weights = []
        for neighbor in neighbors:
            if neighbor == prev_node:
                weights.append(1/self.p)
            elif self.graph.has_edge(curr_node, neighbor):
                weights.append(1)
            else:
                weights.append(1/self.q)
        weights = np.array(weights)
        weights /= weights.sum()
        return np.random.choice(neighbors, p=weights)

    def embed_all_nodes(self):
        embeddings = {}
        for node in self.graph.nodes():
            embeddings[node] = self.model.wv[node]
        return embeddings

    def train_embedding(self):
        model = Word2Vec(
            sentences=self.walks,
            vector_size=self.dimensions,
            window=self.window_size,
            sg=1,  # Skip-gram
            epochs=self.num_epochs,  
            workers=self.workers
        )
        return model

In [ ]:
import numpy as np
import networkx as nx
from gensim.models import Word2Vec

In [ ]:
# # Create a sample graph
# G = nx.Graph()

In [ ]:
# G.add_edges_from([(1, 2), (1, 3), (2, 3), (3, 4), (4, 5), (4, 6), (5, 6),(6,1)])
# G.add_edges_from([(1, 2), (1, 3), (1,4),(1,5), (2, 3), (3, 4), (4, 5), (4, 6), (5, 6),(6,1)])

G.add_edges_from([(1,4),(1,2),(1,3),(1,5),(2,3),(4,6),(4,7),(4,8),(6,8)])


In [ ]:
# Create a random graph with 20 nodes and 40 edges
# G = nx.erdos_renyi_graph(n=20, p=0.3)
G = nx.karate_club_graph()

In [ ]:
# Find the number of nodes and edges
num_nodes = G.number_of_nodes()
num_edges = G.number_of_edges()
print("Number of nodes:", num_nodes)
print("Number of edges:", num_edges)

In [ ]:
G

In [ ]:
# Set random seed for reproducibility
import numpy as np
seed_value = 42
np.random.seed(seed_value)
# List of dimensions to iterate over
dimension_list = [32, 64, 128]

In [ ]:

# # List of dimensions to iterate over
# dimension_list = [32, 64, 128]
# # Iterate over different dimensions
# for dimensions in dimension_list:
#     # Set random seed for Node2Vec and NumPy within each iteration
#     np.random.seed(seed_value)
    
#     # Create the Node2Vec model instance
#     node2vec = Node2Vec(G, p=1, q=0.5, dimensions=dimensions, num_walks=10, walk_length=80, window_size=10, num_epochs=100, workers=4)
    
#     # Train the embedding model using your custom class's train_embedding() method
#     model = node2vec.train_embedding()
    
#     # Embed all nodes
#     node_embeddings = {node: model.wv[node] for node in G.nodes()}
    
#     # Create a new figure and axis
#     fig, ax = plt.subplots(figsize=(8, 6))
    
#     # Visualize the original graph
#     pos = nx.spring_layout(G, seed=seed_value)
#     nx.draw(G, pos, with_labels=True, node_color='skyblue', node_size=500, ax=ax)
#     ax.set_title("Original Graph")
#     plt.show()
    
#     # Plot the embeddings with node numbers
#     plt.figure(figsize=(10, 8))
#     for node, embedding in node_embeddings.items():
#         plt.scatter(embedding[0], embedding[1], s=100)
#         plt.text(embedding[0], embedding[1], str(node), fontsize=10, ha='center', va='center')
#     plt.title(f"Node2Vec Embeddings with Node Numbers (Dimensions={dimensions})")
#     plt.show()

In [ ]:
# Create and train the Node2Vec model
# node2vec = Node2Vec(G, p=1, q=1, dimensions=128, num_walks=10, walk_length=80, window_size=10, num_iter=50, workers=4)
node2vec = Node2Vec(G, p=1, q=0.5, dimensions=32, num_walks=4, walk_length=4, window_size=10, num_epochs=100, workers=4)
node2vec_1 = Node2Vec(G, p=1, q=0.5, dimensions=64, num_walks=4, walk_length=4, window_size=10, num_epochs=100, workers=4)
node2vec_2 = Node2Vec(G, p=1, q=0.5, dimensions=128, num_walks=4, walk_length=4, window_size=10, num_epochs=100, workers=4)


In [ ]:
# Create a figure with subplots
fig, axes = plt.subplots(nrows=len(dimension_list), ncols=2, figsize=(15, 10))

# Iterate over different dimensions
for idx, dimensions in enumerate(dimension_list):
    # Set random seed for Node2Vec and NumPy within each iteration
    np.random.seed(seed_value)
    
    # Create the Node2Vec model instance
    node2vec = Node2Vec(G, p=0.4, q=0.5, dimensions=dimensions, num_walks=10, walk_length=80, window_size=10, num_epochs=100, workers=4)
    
    # Train the embedding model using your custom class's train_embedding() method
    model = node2vec.train_embedding()
    
    # Embed all nodes
    node_embeddings = {node: model.wv[node] for node in G.nodes()}
    
    # Visualize the original graph
    pos = nx.spring_layout(G, seed=seed_value)
    ax = axes[idx, 0]
    nx.draw(G, pos, with_labels=True, node_color='skyblue', node_size=500, ax=ax)
    ax.set_title(f"Original Graph (Dimensions={dimensions})")
    
    # Plot the embeddings with node numbers
    ax = axes[idx, 1]
    for node, embedding in node_embeddings.items():
        ax.scatter(embedding[0], embedding[1], s=100)
        ax.text(embedding[0], embedding[1], str(node), fontsize=10, ha='center', va='center')
    ax.set_title(f"Node2Vec Embeddings (Dimensions={dimensions})")

plt.tight_layout()
plt.show()

In [ ]:
# List of dimensions and walk lengths to iterate over
dimension_list = [32, 64, 128]
walk_length_list = [10, 20, 40]

# Create a figure with subplots
fig, axes = plt.subplots(nrows=len(dimension_list) * len(walk_length_list), ncols=2, figsize=(25, 25))

# Iterate over different dimensions
for d_idx, dimensions in enumerate(dimension_list):
    for w_idx, walk_length in enumerate(walk_length_list):
        # Set random seed for Node2Vec and NumPy within each iteration
        np.random.seed(seed_value)
        
        # Create the Node2Vec model instance
        node2vec = Node2Vec(G, p=0.4, q=0.5, dimensions=dimensions, num_walks=10, walk_length=walk_length, window_size=10, num_epochs=100, workers=4)
        
        # Train the embedding model using your custom class's train_embedding() method
        model = node2vec.train_embedding()
        
        # Embed all nodes
        node_embeddings = {node: model.wv[node] for node in G.nodes()}
        
        # Visualize the original graph
        pos = nx.spring_layout(G, seed=seed_value)
        ax = axes[d_idx * len(walk_length_list) + w_idx, 0]
        nx.draw(G, pos, with_labels=True, node_color='skyblue', node_size=500, ax=ax)
        ax.set_title(f"Original Graph (Dimensions={dimensions}, Walk Length={walk_length})")
        
        # Plot the embeddings with node numbers
        ax = axes[d_idx * len(walk_length_list) + w_idx, 1]
        for node, embedding in node_embeddings.items():
            ax.scatter(embedding[0], embedding[1], s=100)
            ax.text(embedding[0], embedding[1], str(node), fontsize=10, ha='center', va='center')
        ax.set_title(f"Node2Vec Embeddings (Dimensions={dimensions}, Walk Length={walk_length})")

plt.tight_layout()
plt.show()

In [ ]:
# Get the embeddings for a specific node
node_embedding = node2vec.model.wv[1]

In [ ]:
# Iterate through the dictionary and count elements in each array
array_lengths = {}
for key, value in node_embeddings.items():
    array_lengths[key] = len(value)

# Print the counts
for key, length in array_lengths.items():
    print(f"Length of {key}: {length}")

In [ ]:
# Embed all nodes
node_embeddings = node2vec.embed_all_nodes()

In [ ]:
len(node_embeddings)

In [ ]:
# Create a new figure and axis
fig, ax = plt.subplots(figsize=(8, 6))

# Visualize the original graph
pos = nx.spring_layout(G)
nx.draw(G, pos, with_labels=True, node_color='skyblue', node_size=500, ax=ax)
ax.set_title("Original Graph")
plt.show()
plt.close()

In [ ]:
# After embeddings
plt.figure(figsize=(12, 6))
for node, embedding in node_embeddings.items():
    plt.scatter(embedding[0], embedding[1], label=node)
plt.title("Node2Vec Embeddings")
plt.legend()
plt.show()

In [ ]:
# Plot the embeddings with node numbers
plt.figure(figsize=(20, 15))
for node, embedding in node_embeddings.items():
    plt.scatter(embedding[0], embedding[1], s=100)
    plt.text(embedding[0], embedding[1], str(node), fontsize=12, ha='center', va='center')
plt.title("Node2Vec Embeddings with Node Numbers")
plt.show()

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity


In [ ]:
# Calculate cosine similarities between embeddings
embeddings_array = np.array(list(node_embeddings.values()))
cosine_similarities = cosine_similarity(embeddings_array)

In [ ]:
# Plot the cosine similarity heatmap
plt.figure(figsize=(8, 8))
plt.imshow(cosine_similarities, cmap='coolwarm', interpolation='nearest')
plt.colorbar(label='Cosine Similarity')
plt.title("Node2Vec Embeddings Cosine Similarity Heatmap")
plt.show()

In [ ]:
import numpy as np
import networkx as nx
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt

In [ ]:
# List of dimensions and walk lengths to iterate over
dimension_list = [32, 64, 128]
walk_length_list = [10, 20, 40]

# Function to generate node features (replace this with your own feature generation logic)
def generate_features(node):
    # Example: return some numeric features for each node
    return np.random.rand(5)


In [ ]:
# Create a figure with subplots
fig, axes = plt.subplots(nrows=len(dimension_list) * len(walk_length_list), ncols=2, figsize=(15, 15))

# Iterate over different dimensions
for d_idx, dimensions in enumerate(dimension_list):
    for w_idx, walk_length in enumerate(walk_length_list):
        # Set random seed for Node2Vec and NumPy within each iteration
        np.random.seed(seed_value)
        
        # Create the Node2Vec model instance
        node2vec = Node2Vec(G, p=0.4, q=0.5, dimensions=dimensions, num_walks=10, walk_length=walk_length, window_size=10, num_epochs=100, workers=4)
        
        # Train the embedding model using your custom class's train_embedding() method
        model = node2vec.train_embedding()
        
        # Generate node features
        node_features = {node: generate_features(node) for node in G.nodes()}
        
        # Split the graph into training and testing sets
        train_nodes, test_nodes = train_test_split(list(G.nodes()), test_size=0.2, random_state=seed_value)
        
        # Create balanced dataset
        neighbors_exist = [node for node in train_nodes if len(list(G.neighbors(node))) > 0]
        neighbors_not_exist = [node for node in train_nodes if len(list(G.neighbors(node))) == 0]
        
        # Check if there are nodes with neighbors
        if len(neighbors_exist) > 0:
            # Randomly sample from neighbors_exist to balance the dataset
            sampled_neighbors_exist = np.random.choice(neighbors_exist, size=len(neighbors_not_exist), replace=True)
            
            # Combine the balanced samples
            balanced_train_nodes = np.concatenate([sampled_neighbors_exist, neighbors_not_exist])
            
            # Generate labels for the balanced dataset
            y_train = np.array([1 if node in neighbors_exist else 0 for node in balanced_train_nodes])
            
            # Prepare training data
            X_train = np.array([model.wv[node] for node in balanced_train_nodes])

            # Check if X_train is not empty
            if len(X_train) > 0:
                # Train a classifier (Random Forest)
                classifier = RandomForestClassifier(random_state=seed_value)
                classifier.fit(X_train, y_train)
                
                # Prepare testing data
                X_test = np.array([model.wv[node] for node in test_nodes])
                y_test = np.array([1 if node in G.neighbors(node) else 0 for node in test_nodes])
                
                # Evaluate the classifier's performance
                y_pred = classifier.predict(X_test)
                report = classification_report(y_test, y_pred)
                print(f"Classification Report for Dimensions={dimensions}, Walk Length={walk_length}:\n{report}")
                
                # Visualize the original graph
                pos = nx.spring_layout(G, seed=seed_value)
                ax = axes[d_idx * len(walk_length_list) + w_idx, 0]
                nx.draw(G, pos, with_labels=True, node_color='skyblue', node_size=500, ax=ax)
                ax.set_title(f"Original Graph (Dimensions={dimensions}, Walk Length={walk_length})")
                
                # Plot the embeddings with node numbers
                ax = axes[d_idx * len(walk_length_list) + w_idx, 1]
                for node, embedding in node_embeddings.items():
                    ax.scatter(embedding[0], embedding[1], s=100)
                    ax.text(embedding[0], embedding[1], str(node), fontsize=10, ha='center', va='center')
                ax.set_title(f"Node2Vec Embeddings (Dimensions={dimensions}, Walk Length={walk_length})")
            else:
                print("No embeddings for nodes in the training dataset.")
        else:
            print("No nodes with neighbors in the training dataset.")

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import accuracy_score

import os
import networkx as nx
import numpy as np
import pandas as pd

from stellargraph.data import BiasedRandomWalk
from stellargraph import StellarGraph
from stellargraph import datasets
from IPython.display import display, HTML

%matplotlib inline

In [ ]:
type(G)

In [ ]:
G

In [ ]:
num_nodes = G.number_of_nodes()
num_edges = G.number_of_edges()

print("Number of nodes:", num_nodes)
print("Number of edges:", num_edges)

In [ ]:
# Convert the NetworkX graph to a StellarGraph object
stellar_graph = StellarGraph.from_networkx(G)

In [ ]:
print(stellar_graph.info())

In [ ]:
rw = BiasedRandomWalk(stellar_graph)

In [ ]:
walks = rw.run(
    nodes=list(stellar_graph.nodes()),  # root nodes
    length=100,  # maximum length of a random walk
    n=10,  # number of random walks per root node
    p=0.5,  # Defines (unormalised) probability, 1/p, of returning to source node
    q=2.0,  # Defines (unormalised) probability, 1/q, for moving away from source node
)
print("Number of random walks: {}".format(len(walks)))

In [ ]:
from gensim.models import Word2Vec

str_walks = [[str(n) for n in walk] for walk in walks]
# model = Word2Vec(str_walks, size=128, window=5, min_count=0, sg=1, workers=2, iter=1)
model = Word2Vec(sentences=str_walks, vector_size=128, window=5, min_count=0, workers=4,epochs=100)

In [ ]:
model.wv["1"].shape

In [ ]:
# Retrieve node embeddings and corresponding subjects
node_ids = model.wv.index_to_key  # list of node IDs
node_embeddings = (
    model.wv.vectors
)  # numpy.ndarray of size number of nodes times embeddings dimensionality
# node_targets = node_subjects[[int(node_id) for node_id in node_ids]]

In [ ]:
# dataset = datasets.Cora()
# display(HTML(dataset.description))
# G_1, node_subjects = dataset.load(largest_connected_component_only=True)

In [ ]:
# Apply t-SNE transformation on node embeddings
tsne = TSNE(n_components=2)
node_embeddings_2d = tsne.fit_transform(node_embeddings)

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
# Use KMeans to cluster nodes into subjects
num_clusters = 5  # Number of clusters (subjects)
kmeans = KMeans(n_clusters=num_clusters, random_state=0)
node_targets = kmeans.fit_predict(node_embeddings)

In [ ]:
node_targets

In [ ]:
# draw the points
alpha = 0.7
label_map = {l: i for i, l in enumerate(np.unique(node_targets))}
node_colours = [label_map[target] for target in node_targets]

plt.figure(figsize=(10, 8))
plt.scatter(
    node_embeddings_2d[:, 0],
    node_embeddings_2d[:, 1],
    c=node_colours,
    cmap="jet",
    alpha=alpha,
)

In [ ]:
# X will hold the 128-dimensional input features
X = node_embeddings
# y holds the corresponding target values
y = np.array(node_targets)

In [ ]:
X

In [ ]:
y

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, test_size=None)

In [ ]:
len(X_test)

In [ ]:
print(
    "Array shapes:\n X_train = {}\n y_train = {}\n X_test = {}\n y_test = {}".format(
        X_train.shape, y_train.shape, X_test.shape, y_test.shape
    )
)

In [ ]:
clf = LogisticRegressionCV(
    Cs=5, cv=5, scoring="accuracy", verbose=True, multi_class="ovr", max_iter=300
)
clf.fit(X_train, y_train)

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred)